# install the packages



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import clear_output
!pip install twarc #Twarc
!pip install tweepy # Tweepy 3.8.0
!pip install argparse #Argparse 3.2
!pip install xtract #Xtract 0.1 a3
!pip install wget #Wget 3.2
clear_output()

In [ ]:
import gzip
import shutil
import os
import wget
import csv
import linecache
from shutil import copyfile
import ipywidgets as widgets
import numpy as np
import pandas as pd

In [ ]:
import json
import tweepy
from tweepy import OAuthHandler

# Authenticate
CONSUMER_KEY = "J9rvfuIoUYXukMaxd58lJw8ZQ" #@param {type:"string"}
CONSUMER_SECRET_KEY = "CYtX7T7cpIDkG69qxECXq3ObRUwxDVqeSRuwlA4ggVShlRhmtR" #@param {type:"string"}
ACCESS_TOKEN_KEY = "1476442025268752388-EEOH89okpXx1WJOVhewOqHYfG1XzFQ" #@param {type:"string"}
ACCESS_TOKEN_SECRET_KEY = "hHvdGJuuNerww38p1hUJBRLMUfRyBEjddjqyFXHFdI3yw" #@param {type:"string"}

#Creates a JSON Files with the API credentials
with open('api_keys.json', 'w') as outfile:
    json.dump({
    "consumer_key":CONSUMER_KEY,
    "consumer_secret":CONSUMER_SECRET_KEY,
    "access_token":ACCESS_TOKEN_KEY,
    "access_token_secret": ACCESS_TOKEN_SECRET_KEY
     }, outfile)


In [ ]:
from IPython.display import clear_output
import nltk

nltk.download('stopwords')
nltk.download("punkt")
nltk.download('wordnet')

!wget https://raw.githubusercontent.com/thepanacealab/SMMT/master/data_preprocessing/parse_json_lite.py -O parse_json_lite.py
!wget https://raw.githubusercontent.com/thepanacealab/SMMT/master/data_preprocessing/fields.py -O fields.py

!pip install emot --upgrade
!pip install emoji --upgrade
!pip install autocorrect
!pip install langdetect

clear_output()

# Filter English

In [ ]:
# Filtering first
import pandas as pd
import numpy as np
import json
import sys
import re
import string
# This will load the fields list
import fields
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from langdetect import detect
from collections import Counter

fieldsFilter = fields.fields
fileN = '/content/drive/MyDrive/covid analysis/hydrated_1/hydrated_tweets_short_0327.json'
data = []

with open(fileN, 'r') as f:
    for line in f:
        data.append(json.loads(line))

def detect_lang(text):
    try :    
      return detect(text)
    except :
      return np.nan

tweet_df = pd.json_normalize(data)
# Cleaner solution in case some of the fields in the list are non existent and/or have typos
tweet_df = tweet_df.loc[:, tweet_df.columns.isin(fieldsFilter)]

tw = pd.DataFrame(columns=["created_at", "text", "lang"], index=range(0, len(tweet_df)))
for j in range(len(tweet_df['text'])):
    tw.loc[j]['created_at'] = tweet_df.loc[j]['created_at']
    tw.loc[j]['text'] = tweet_df.loc[j]['text']
    tw.loc[j]['lang'] = detect_lang(tw.loc[j]['text'])
print(tw.head())

KeyboardInterrupt: ignored

In [ ]:
#Creates a new clean dataset with the specified language (if specified)
filtered_language = 'en'
cnt = 0
en_tw = pd.DataFrame(columns=["created_at", "text"], index=range(0, len(tw)))
for i in range(len(tw['text'])):
  if tw.loc[i]['lang'] == filtered_language:
    en_tw.loc[cnt]['created_at'] = tw.loc[i]['created_at']
    en_tw.loc[cnt]['text'] = tw.loc[i]['text']
    cnt += 1

en_tw = en_tw.dropna(axis=0)
print(en_tw.head())
print(len(en_tw))
with open("english-0326"+".json",'w') as write_json:
  write_json.write(en_tw.to_json(orient = 'records', lines=True))

# Filter by keywords

In [ ]:
en_tw['text'] = en_tw['text'].str.lower()
filtered = pd.DataFrame(columns=["created_at", "text"], index=range(0, 5000000))

#filtering with keywords
keywords = ['covid19', 'covid-19', 'corona', 'coronavirus', 'virus', 'pandemic', 'outbreak', 'wuhan']
cnt = 0
for j in range(len(en_tw)):
  temp = en_tw.loc[j]['text']
  for i in keywords :
    if i in temp :
      filtered.loc[cnt]['created_at'] = en_tw.loc[j]['created_at']
      filtered.loc[cnt]['text'] = temp
      cnt += 1
    continue
    
filtered = filtered.dropna(axis=0)
filtered = filtered.drop_duplicates()
print(len(filtered))

with open("filtered_0326"+".json",'w') as write_json:
  write_json.write(filtered.to_json(orient = 'records', lines=True))

# Preprocessing

In [ ]:
# Filtering first
import pandas as pd
import numpy as np
import json
import sys
import string
import re
import fields
from emot.emo_unicode import EMOJI_UNICODE, EMOTICONS_EMO
import emoji
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from collections import Counter
from autocorrect import Speller
from textblob import TextBlob
try:
    import cPickle as pickle
except ImportError: 
    import pickle

In [ ]:
fieldsFilter = fields.fields
fileN = '/content/drive/MyDrive/covid analysis/filtered_2/filtered_1005.json'

data = []

with open(fileN, 'r') as f:
    for line in f:
        data.append(json.loads(line))

tweet_new = pd.json_normalize(data)
tweet_new = tweet_new.loc[:, tweet_new.columns.isin(fieldsFilter)]
tweet_new['index'] = tweet_new.index

* Treat Emoji and Emoticons

In [ ]:
emoticon_file = '/content/drive/MyDrive/covid analysis/Emoticon_Dict.p'
emoji_file = '/content/drive/MyDrive/covid analysis/Emoji_Dict.p'

with open(emoticon_file, 'rb') as fp:
    Emoticon_Dict = pickle.load(fp)

with open(emoji_file, 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emoticons(text):
    for emot in Emoticon_Dict:
        text = str(text)
        text = text.replace(emot, "_".join(Emoticon_Dict[emot].replace(",","").replace(":","").split()))
        return text

def convert_emojis(text):
    for emot in Emoji_Dict:
        text = str(text)
        text = text.replace(emot, "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()))
        return text

def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)


# lowercasing
tweet_new['text'] = tweet_new['text'].str.lower()

# convert emoji and emoticons
tweet_new['text'] = tweet_new['text'].apply(lambda x : convert_emoticons(x))
tweet_new['text'] = tweet_new['text'].apply(lambda x : convert_emojis(x))
tweet_new['text'] = tweet_new['text'].apply(lambda x : give_emoji_free_text(x))


In [ ]:
print(len(tweet_new['text']))

48317


* Remove noise

In [ ]:
def remove_urls(text):
    result = re.sub(r"http\S+", "", text)
    return(result)

def remove_twitter_urls(text):
    clean = re.sub(r"pic.twitter\S+", "",text)
    return(clean)

def remove_mentions(text):
    mentions = re.compile(r"@\w+ ?")
    return mentions.sub(r"", text)

def remove_numbers(text):
    return re.sub(r"\d+", "", text)

def remove_others(text) :
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
# remove noise
tweet_new['text'] = tweet_new['text'].apply(lambda x : remove_urls(x))
tweet_new['text'] = tweet_new['text'].apply(lambda x : remove_twitter_urls(x))
tweet_new['text'] = tweet_new['text'].apply(lambda x : remove_mentions(x))
tweet_new['text'] = tweet_new['text'].apply(lambda x : remove_numbers(x))
tweet_new['text'] = tweet_new['text'].apply(lambda x : remove_others(x))
print(len(tweet_new['text']))

48317


* For sentimentr - score sentiment

In [ ]:
punc_list = string.punctuation + '…' + '—' + '“' + '”' + '‘' + '’' + '•' + '↓' 
def remove_punctuations(text):
    translator = str.maketrans("", "", punc_list)
    return str(text).translate(translator)

def non_english(text) :
    text = re.sub(r'\W+', ' ', text)
    text = text.replace("[^a-zA-Z]", " ")
    word_tokens = word_tokenize(text)
    filtered_word = [w for w in word_tokens if all(ord(c) < 128 for c in w)]
    filtered_word = [w + " " for w in filtered_word]
    return "".join(filtered_word)

tweet_new['text'] = tweet_new['text'].apply(lambda x : remove_punctuations(x))
tweet_new['text'] = tweet_new['text'].apply(lambda x : non_english(x))
print(len(tweet_new['text']))

48317


In [ ]:
tweet_new.head()

,created_at,text,index
0,Tue Oct 05 04:01:49 +0000 2021,india records new covid cases in past hours lo...,0
1,Tue Oct 05 04:01:48 +0000 2021,jann arden says while covid is very real it wo...,1
2,Tue Oct 05 04:01:50 +0000 2021,jann arden says while covid is very real it wo...,2
3,Tue Oct 05 04:02:17 +0000 2021,pdf download and the people stayed home family...,3
4,Tue Oct 05 04:02:13 +0000 2021,trying to understand why usa got hit so hard d...,4


In [ ]:
#tweet_new = tweet_new.drop(['created_at'], axis = 'columns')
tweet_new.to_csv('/content/drive/MyDrive/covid analysis/sentence_2/1005.csv', columns = ['text', 'index'], header = True, index = False)

* Proceed NLP tasks

In [ ]:
all_stopwords = stopwords.words('english')
all_stopwords.extend(['via', 'amp', 'due', 'one', 'nt'])
print(all_stopwords)
def stop_words(text):
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in all_stopwords]
    return " ".join(tokens_without_sw)

punc_list = string.punctuation + '…' + '—' + '“' + '”' + '‘' + '’' + '•' + '↓' 
def remove_punctuations(text):
    translator = str.maketrans("", "", punc_list)
    return str(text).translate(translator)

def non_english(text) :
    text = re.sub(r'\W+', ' ', text)
    text = text.replace("[^a-zA-Z]", " ")
    word_tokens = word_tokenize(text)
    filtered_word = [w for w in word_tokens if all(ord(c) < 128 for c in w)]
    filtered_word = [w + " " for w in filtered_word]
    return "".join(filtered_word)

keywords = ['covid', 'corona', 'coronavirus', 'virus', 'pandemic', 'outbreak', 'wuhan']
def remove_keywords(text):
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in keywords]
    return tokens_without_sw

def remove_onewords(text):
    correct = []
    for word in text :
      if len(word) >= 2 :
        correct.append(word)
    return correct

ps = PorterStemmer()
def stem_text(text):
  stem = []
  for word in text :
    stem.append(ps.stem(word))
  return stem 

lemmatizer = WordNetLemmatizer()
def lem_text(text):
  lemm = []
  for word in text :
    lemm.append(lemmatizer.lemmatize(word))
  return lemm

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
print(tweet_new['text'].head(10))
print(tweet_new['text'].tail(10))
tweet_new['text'] = tweet_new['text'].apply(lambda x : stop_words(x))
print(len(tweet_new['text']))
tweet_new['text'] = tweet_new['text'].apply(lambda x : remove_keywords(x))
print(len(tweet_new['text']))
tweet_new['text'] = tweet_new['text'].apply(lambda x : remove_onewords(x))
print(len(tweet_new['text']))
tweet_new['text'] = tweet_new['text'].apply(lambda x : stem_text(x))
print(len(tweet_new['text']))
tweet_new['text'] = tweet_new['text'].apply(lambda x : lem_text(x))
print(tweet_new['text'].head(10))
print(tweet_new['text'].tail(10))
print(len(tweet_new['text']))

0    india records new covid cases in past hours lo...
1    jann arden says while covid is very real it wo...
2    jann arden says while covid is very real it wo...
3    pdf download and the people stayed home family...
4    trying to understand why usa got hit so hard d...
5    time is running out for schools to apply for f...
6    its a pandemic of the unvaccinated right how a...
7    since the start of the pandemic americans have...
8    covid cases among children are rising signific...
9    have you received your covid vaccination us ci...
Name: text, dtype: object
48307    covid india reports new cases in the last hour...
48308    potential for false positive results with cert...
48309    south africa launches digital covid vaccinatio...
48310    what an idiot i love the astros but they wont ...
48311    remember all vaccines are effective in prevent...
48312    the big question is is your government health ...
48313    since the start of the pandemic americans have...
48314    co

In [ ]:
#tweet_new = tweet_new.drop(['created_at'], axis = 'columns')
tweet_new.to_csv('/content/drive/MyDrive/covid analysis/preprocessed_2/1005.csv', columns = ['text', 'index'], header = True, index = False)

# Compute uni and n grams

In [ ]:
import csv 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 

def concat(text):
  return " ".join([word for word in str(text).split()])

tweet_new['text'] = tweet_new['text'].apply(lambda x : concat(x))

# Getting ngrams  
vectorizer = CountVectorizer(ngram_range = (1,1)).fit(tweet_new["text"]) 

bag_of_words = vectorizer.transform(tweet_new["text"])
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

with open("1005-1gram.csv", "w") as f:
    w = csv.writer(f)
    w.writerows(words_freq)

In [ ]:
import csv 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 

def concat(text):
  return " ".join([word for word in str(text).split()])

tweet_new['text'] = tweet_new['text'].apply(lambda x : concat(x))

# Getting ngrams  
vectorizer = CountVectorizer(ngram_range = (2,3)).fit(tweet_new["text"]) 

bag_of_words = vectorizer.transform(tweet_new["text"])
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

with open("1005-ngram.csv", "w") as f:
    w = csv.writer(f)
    w.writerows(words_freq)